In [1]:
import rasterio
import dask.array as da
from dask_rasterio import read_raster, write_raster
import os
import fnmatch
import numpy as np
import geopandas as gpd
import shapely
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy.spatial as es
import seaborn as sns
import matplotlib.pyplot as plt
import earthpy.plot as ep
import imageio


In [2]:
def projectRaster(raster_fn, raster_src, projected_dir, dst_crs):
    transform, width, height = calculate_default_transform(
            raster_src.crs, dst_crs, raster_src.width, raster_src.height, *raster_src.bounds)
    kwargs = raster_src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    projected_raster = projected_dir + '\\' + raster_fn.split("\\")[-1]
    with rasterio.open(projected_raster, 'w', **kwargs) as dst:
        for i in range(1, raster_src.count + 1):
            reproject(
                source=rasterio.band(raster_src, i),
                destination=rasterio.band(dst, i),
                src_transform=raster_src.transform,
                src_crs=raster_src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
    return projected_raster, kwargs

In [3]:
def crop_raster(src, crop_shp):
    cropped_raster, cropped_meta = es.crop_image(src, crop_shp)
    array = cropped_raster
    array[array==0] = np.nan    
    array[array==9999] = np.nan
    return array, cropped_meta

In [3]:
def cropRasterByMask(src_projected, crop_shp, cropped_raster_fn):
    geom = []
    coord = shapely.geometry.mapping(crop_shp)["features"][0]["geometry"]
    geom.append(coord)

    out_image, out_transform = rasterio.mask.mask(src_projected, geom, crop=True)
    out_meta = src_projected.meta
    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})

    with rasterio.open(os.path.join(cropped_raster_fn), "w", **out_meta) as dest:
        dest.write(out_image)

In [4]:
input_dir = r'Z:\LWI_StageIV\Hurricane Katrina 2005'
output_dir= r'Z:\LWI_StageIV\!Accumulated'
crop_shp = gpd.read_file(r"Z:\GIS\StageIv Boundary.shp")
dst_crs = 'EPSG:4326' 
projected_dir = os.path.join(input_dir, 'projected')
cropped_dir = os.path.join(input_dir, 'cropped')

# Assumes Directory Name is the name of the storm and wanted name of the output files.
# I.e: input_dir=r'Z:\LWI_StageIV\Hurricane Cindy 2005' ==> output file will be 'Hurricane Cindy 2005.tif'
outputFilename = input_dir.split('\\')[-1]+'-cropped.tif'

# Check if output, projected, and cropped directories exist.
isExist = os.path.exists(output_dir)
if not isExist:
        os.makedirs(output_dir)
isExist = os.path.exists(projected_dir)
if not isExist:
        os.makedirs(projected_dir)
isExist = os.path.exists(cropped_dir)
if not isExist:
        os.makedirs(cropped_dir)

# get all files with extension .01h (the default pattern for uncompressed 1hr precip data from UCAR EOL).
# Script is not accounting for potential naming patterns for AK and PR.
merge_dir=[]
for filename in fnmatch.filter(os.listdir(input_dir),'*.01h'): 
    merge_dir.append((os.path.join(input_dir, filename)))

merge_dir.sort()

map2array=[]
for raster in merge_dir:
    print(f'projecting and cropping {raster}. Then adding to dask array list: map2array')
    src = rasterio.open(raster)
    projected_raster, kwargs = projectRaster(raster, src, projected_dir, dst_crs)
    src_projected = rasterio.open(projected_raster)
    # array, cropped_meta = crop_raster(src, crop_shp)
    # Write Cropped Raster to Disk
    cropped_raster_fn = cropped_dir + '\\' + raster.split("\\")[-1]
    cropRasterByMask(src_projected, crop_shp, cropped_raster_fn)
    cropped_src = rasterio.open(cropped_raster_fn)
    profile = cropped_src.profile
    array = cropped_src.read(1)
    array[array==0] = np.nan    
    array[array==9999] = np.nan
    # with rasterio.open(cropped_raster_fn, 'w', **kwargs) as dest:
    #     dest.write(array.squeeze().astype(rasterio.uint8), 1)
    #     profile = dest.profile
    # map2array.append(read_raster(raster, band=1, block_size=10))
    map2array.append(da.from_array(array, chunks=array.shape))
    src.close()
    src_projected.close()
    cropped_src.close()
# with rasterio.open(merge_dir[0]) as src:
#     profile=src.profile




projecting and cropping Z:\LWI_StageIV\Hurricane Katrina 2005\ST4.2005082000.01h. Then adding to dask array list: map2array
projecting and cropping Z:\LWI_StageIV\Hurricane Katrina 2005\ST4.2005082001.01h. Then adding to dask array list: map2array
projecting and cropping Z:\LWI_StageIV\Hurricane Katrina 2005\ST4.2005082002.01h. Then adding to dask array list: map2array
projecting and cropping Z:\LWI_StageIV\Hurricane Katrina 2005\ST4.2005082003.01h. Then adding to dask array list: map2array
projecting and cropping Z:\LWI_StageIV\Hurricane Katrina 2005\ST4.2005082004.01h. Then adding to dask array list: map2array
projecting and cropping Z:\LWI_StageIV\Hurricane Katrina 2005\ST4.2005082005.01h. Then adding to dask array list: map2array
projecting and cropping Z:\LWI_StageIV\Hurricane Katrina 2005\ST4.2005082006.01h. Then adding to dask array list: map2array
projecting and cropping Z:\LWI_StageIV\Hurricane Katrina 2005\ST4.2005082007.01h. Then adding to dask array list: map2array
projecti

In [5]:
ds_stack = da.stack(map2array)

# with rasterio.open(merge_dir[0]) as src:
#     profile=src.profile
#     profile.update(compress='lzw')
print (f'writing Raster to {output_dir}\\{outputFilename}')
write_raster(f'{output_dir}\\{outputFilename}', da.nansum(ds_stack,0), **profile)
# write_raster(f'{output_dir}\\{outputFilename}', da.nansum(ds_stack,0))

writing Raster to Z:\LWI_StageIV\!Accumulated\Hurricane Katrina 2005-cropped.tif


In [ ]:
raster_fn = r'Z:\LWI_StageIV\test\ST4.2005082000.01h'
crop_shp = gpd.read_file(r"Z:\GIS\StageIv Boundary.shp")
dst_crs = 'EPSG:4326' 
input_dir = r'Z:\LWI_StageIV\test'
projected_dir = os.path.join(input_dir, 'projected')

src = rasterio.open(raster_fn)
projected_raster, kwargs = projectRaster(raster_fn, src, projected_dir, dst_crs)
src_projected = rasterio.open(projected_raster)
# array, cropped_meta = crop_raster(src, crop_shp)
# Write Cropped Raster to Disk
# cropped_raster_fn = cropped_dir + '\\' + raster.split("\\")[-1]
# with rasterio.open(cropped_raster_fn, 'w', **kwargs) as dest:
#     dest.write(array.squeeze().astype(rasterio.uint8), 1)
#     profile = dest.profile
# map2array.append(da.from_array(array, chunks=array.shape))

# src.close()
# src_projected.close()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
la = gpd.read_file("Z:\GIS\Louisiana.shp")
bbox = la.total_bounds
la_extent=bbox[[0,2,1,3]]

bbox_lwi = crop_shp.total_bounds
lwi_extent = bbox_lwi[[0,2,1,3]]
fn = raster_fn
title = fn.split('\\')[-1]
raster = src_projected

# raster.window?
raster_window = raster.window(*bbox_lwi)
array = raster.read(1, window=raster_window)
array[array==raster.nodata] = np.nan
im = plt.imshow(array, extent=lwi_extent, cmap='rainbow')
cb = plt.colorbar(im, shrink=.6)
ax.set(title=f"{title} Cumulative Precip (mm)")
ax.set_axis_off()
la.boundary.plot(ax=plt.gca(), color='darkgrey')

In [ ]:
import shapely
geom = []
coord = shapely.geometry.mapping(crop_shp)["features"][0]["geometry"]
geom.append(coord)

out_image, out_transform = rasterio.mask.mask(src_projected, geom, crop=True)
out_meta = src_projected.meta
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open("test.tif", "w", **out_meta) as dest:
    dest.write(out_image)